In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pysabr import Hagan2002LognormalSABR
from pysabr import hagan_2002_lognormal_sabr as sabr

In [ ]:
# calibrate all historic sabr parameters

In [3]:
swaption=pd.read_csv("swaption1.csv")
swaption['Date']=pd.to_datetime(swaption['Date'],format='%Y-%m-%d')
swaption=swaption.drop('Unnamed: 0',axis=1)
unique_combos = swaption[['Expiry', 'Tenor']].drop_duplicates()

In [4]:
date=swaption['Date'].drop_duplicates()
date=date.reset_index(drop=True)

In [7]:
unique_combos=unique_combos.reset_index(drop=True)
unique_combos

In [ ]:
unique_combos.to_csv('unique_combo.csv')

In [9]:
swaption['alpha'] = ''
swaption['rho'] = ''
swaption['volvol'] = ''
swaption['sabrvol']=''
swaption['error']=''
swaption['abs(e_size)>0.005']=''

In [10]:
swaption

,Date,Expiry,Tenor,Strike,Vols,Forward Swap Rate,Numerical Strike,alpha,rho,volvol,sabrvol,error,abs(e_size)>0.005
0,2021-01-13,10.0,10.0,atm-1.0%,0.170552,1.656352,1.639789,,,,,,
1,2021-01-13,10.0,10.0,atm-0.5%,0.135412,1.656352,1.648071,,,,,,
2,2021-01-13,10.0,10.0,atm,0.114587,1.656352,1.656352,,,,,,
3,2021-01-13,10.0,10.0,atm+0.5%,0.137742,1.656352,1.664634,,,,,,
4,2021-01-13,10.0,10.0,atm+1.0%,0.169685,1.656352,1.672916,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
315895,2024-01-12,8.0,8.0,atm-1.0%,0.259523,3.440227,3.405825,,,,,,
315896,2024-01-12,8.0,8.0,atm-0.5%,0.243703,3.440227,3.423026,,,,,,
315897,2024-01-12,8.0,8.0,atm,0.239538,3.440227,3.440227,,,,,,
315898,2024-01-12,8.0,8.0,atm+0.5%,0.262953,3.440227,3.457428,,,,,,


In [54]:
for i in range(len(unique_combos)):
    a,b=unique_combos.loc[i]
    for j in range(len(date)):
        d=date[j]  
        test=swaption[(swaption['Date']==d)&(swaption['Expiry']==a)&(swaption['Tenor']==b)].reset_index(drop=True)
        f=test.loc[0,'Forward Swap Rate']
        k=np.array(test['Numerical Strike'])
        Vol=np.array(test['Vols'])
        t=test.loc[0,'Expiry']
        beta=0.5
        sabr_lognormal = Hagan2002LognormalSABR(f=f,t=t,beta=beta)
        alpha, rho, volvol = sabr_lognormal.fit(k, Vol)
        calibrated_vols = [sabr.lognormal_vol(strike, f, t, alpha, beta, rho, volvol) * 100 for strike in k]
        swaption.loc[((swaption['Expiry']==a)&(swaption['Tenor']==b)&(swaption['Date']==d)),'alpha']=alpha
        swaption.loc[((swaption['Expiry']==a)&(swaption['Tenor']==b)&(swaption['Date']==d)),'rho']=rho
        swaption.loc[((swaption['Expiry']==a)&(swaption['Tenor']==b)&(swaption['Date']==d)),'volvol']=volvol
        swaption.loc[((swaption['Expiry']==a)&(swaption['Tenor']==b)&(swaption['Date']==d)),'sabrvol']=calibrated_vols
        error=Vol-calibrated_vols
        swaption.loc[((swaption['Expiry']==a)&(swaption['Tenor']==b)&(swaption['Date']==d)),'error']=error
        swaption.loc[((swaption['Expiry']==a)&(swaption['Tenor']==b)&(swaption['Date']==d)),'abs(e_size)>0.005']=abs(error)>0.005
#     swaption[(swaption['Expiry']==a)&(swaption['Tenor']==b)]

In [55]:
swaption.to_csv('sabr.csv') # export swaption dataframe with sabr parameters